In [1]:
import pyger
import numpy as np
import matplotlib.pyplot as plt
from Chandra.Time import DateTime
import cPickle as pickle
import numpy as np

Note that the DH on and off options are both used automatically

In [12]:
def assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates):

    template = {'max_1dpamzt': '999',
                'max_1deamzt': '999',
                'max_1pdeaat': '999',
                'max_aacccdpt': '999',
                'max_pftank2t': '999',
                'max_tcylaft6': '999',
                'max_tephin': '999',
                'max_4rt700t': '999',
                'max_fptemp_11': '999'}

    newcases = []
    for date in dates:
        for msid in options.keys():
            lims = np.arange(options[msid]['startlim'], options[msid]['stoplim'] + options[msid]['limstep'], options[msid]['limstep'])
            for lim in lims:
                if options[msid]['sameccd'] == True:
                    nccds = zip(options[msid]['nccd1'], options[msid]['nccd1'])
                else:
                    nccds = list(itertools.product(options[msid]['nccd1'], options[msid]['nccd2']))

                if 'cool_pitch_min' in options[msid].keys():
                    coolpitchmin = options[msid]['cool_pitch_min']
                    coolpitchmax = options[msid]['cool_pitch_max']
                else:
                    coolpitchmin = None
                    coolpitchmax = None
                    
                roll = options[msid.upper()]['roll'] 

                for nccd1, nccd2 in nccds:
                    for dh in options[msid]['dh']:
                        s = template.copy()
                        s['msid'] = msid
                        s['constraint_model'] = options[msid.upper()]['model']
                        s['filename'] = 'pyger_single_msid_{}{}_{}_{}_{}ccd-{}ccd_DH-{}_roll-{}'.format(date[:4], date[-3:],
                                                                                                        msid.lower(), lim,
                                                                                                        nccd1, nccd2, dh, roll)
#                         s['filename'] = 'pyger_single_msid_{}{}_{}_{}_{}ccd_DH-{}'.format(date[:4], date[-3:], msid.lower(), lim, nccd1, dh)
                        s['max_' + msid.lower()] = str(lim)
                        s['title'] = msid.upper() + ': ' + str(lim) + options[msid]['units'] + ' ' + date
                        s['n_ccd_dwell1'] = str(nccd1)
                        s['n_ccd_dwell2'] = str(nccd2)
                        s['dh_heater'] = str(dh == 'ON')
                        s['start'] = date
                        s['n_sim'] = str(n_sim)
                        s['max_dwell_ksec'] = max_dwell_ksec
                        s['dwell_2_pitch_num'] = str(dwell_2_pitch_num)
                        s['cool_pitch_min'] = coolpitchmin
                        s['cool_pitch_max'] = coolpitchmax
                        s['roll'] = roll                    
                        newcases.append(s)

    return newcases

def write_cases(newcases, filename):
    fid = open(filename, 'w')
    
    header = newcases[0].keys()
    
    # Write Header
    for name in header[:-1]:
        fid.write('{},'.format(name))
    fid.write('{}\n'.format(header[-1]))

    # Write Cases
    for case in newcases:
        for name in header[:-1]:
            fid.write('{},'.format(case[name]))
        fid.write('{}\n'.format(case[header[-1]]))
    
    fid.close()
    
    
def chunks(listofstuff, n):
    """Yield successive n-sized chunks from l."""
    for i in xrange(0, len(listofstuff), n):
        yield listofstuff[i:i+n]
    
    
def factors(n):    
    return set(reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))

In [5]:
#header = ('title', 'filename', 'start', 'max_tephin', 'max_tcylaft6', 'max_1pdeaat', 
#              'max_1dpamzt', 'max_pftank2t', 'max_aacccdpt', 'n_ccd_dwell1', 'n_ccd_dwell2', 
#              'n_sim', 'dwell_2_pitch_num', 'max_dwell_ksec', 'constraint_model', 'msid')
    
options = {'TCYLAFT6':{'units':'F', 'startlim':108, 'stoplim':120, 'limstep':1, 'nccd1':[4,], 'dh':['ON'], 'model':'tcylaft6', 'sameccd': True, 'roll': 0.0},
           'PFTANK2T':{'units':'F', 'startlim':93, 'stoplim':97, 'limstep':1, 'nccd1':[4,], 'dh':['ON'], 'model':'tank', 'sameccd': True, 'roll': 0.0},
           'AACCCDPT':{'units':'C', 'startlim':-15, 'stoplim':-5, 'limstep':1, 'nccd1':[4,], 'dh':['ON'], 'model':'aca', 'sameccd': True, 'roll': 0.0},
           '4RT700T':{'units':'F', 'startlim':82, 'stoplim':90, 'limstep':1, 'nccd1':[4,], 'dh':['ON'], 'model':'fwdblkhd', 'sameccd': True, 'roll': 0.0},
           '1PDEAAT':{'units':'C', 'startlim':52.5, 'stoplim':53, 'limstep':0.5, 'nccd1':[1, 3, 4, 5, 6], 'dh':['ON', 'OFF'], 'model':'psmc', 'sameccd': True, 'roll': 0.0},
           '1DPAMZT':{'units':'C', 'startlim':35, 'stoplim':38, 'limstep':0.5, 'nccd1':[1, 3, 4, 5, 6], 'dh':['ON', 'OFF'], 'model':'dpa', 'sameccd': True, 'roll': 0.0},
           '1DEAMZT':{'units':'C', 'startlim':35, 'stoplim':38, 'limstep':0.5, 'nccd1':[1, 3, 4, 5, 6], 'dh':['ON', 'OFF'], 'model':'dea', 'sameccd': True, 'roll': 0.0}}

n_sim = 2000
max_dwell_ksec = 700
dwell_2_pitch_num = 200


In [6]:
dates = [year + ':' + day for day in ['001', '090'] for year in ['2016']]
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
write_cases(newcases, 'cases_2016_001_090.csv')
   
dates = [year + ':' + day for day in ['182', '274'] for year in ['2016']]
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
write_cases(newcases, 'cases_2016_182_274.csv')

dates = [year + ':' + day for day in ['001', '090'] for year in ['2017']]
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
write_cases(newcases, 'cases_2017_001_090.csv')
   
dates = [year + ':' + day for day in ['182', '274'] for year in ['2017']]
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
write_cases(newcases, 'cases_2017_182_274.csv')

dates = [year + ':' + day for day in ['001', '090'] for year in ['2018']]
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
write_cases(newcases, 'cases_2018_001_090.csv')
   
dates = [year + ':' + day for day in ['182', '274'] for year in ['2018']]
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
write_cases(newcases, 'cases_2018_182_274.csv')


In [7]:
len(newcases)

396

Use the following code (uncommented), either in a notebook or in an iPython text based window, to run each set of cases assembled above.

In [ ]:
cases = pyger.read_cases('cases_set1.csv')
#pyger.run_pyger_cases(cases)

Use the following code to print out a custom list of files to break up running a model over several processes.

In [8]:
n_sim = 2000
max_dwell_ksec = 700
dwell_2_pitch_num = 200

dates = [year + ':' + day for day in ['001', '090', '182', '274'] for year in ['2016', ]]
options = {'FPTEMP_11':{'units':'C', 'startlim':-119, 'stoplim':-114, 'limstep':5, 'nccd1':[1,4,5,6],
                        'nccd2':[1,], 'dh':['ON', 'OFF'], 'model':'acisfp', 'sameccd':False,
                        'cool_pitch_min':45, 'cool_pitch_max':169}}
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
options = {'FPTEMP_11':{'units':'C', 'startlim':-119, 'stoplim':-114, 'limstep':5, 'nccd1':[4,5,6],
                        'dh':['ON', 'OFF'], 'model':'acisfp', 'sameccd':True}}
newcases.extend(assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates))
casechunks = list(chunks(newcases, 14))
for n, cases in enumerate(casechunks):
    write_cases(cases, 'cases_acis_2016-{}.csv'.format(n))

dates = [year + ':' + day for day in ['001', '090', '182', '274'] for year in ['2017', ]]
options = {'FPTEMP_11':{'units':'C', 'startlim':-119, 'stoplim':-114, 'limstep':5, 'nccd1':[1,4,5,6],
                        'nccd2':[1,], 'dh':['ON', 'OFF'], 'model':'acisfp', 'sameccd':False,
                        'cool_pitch_min':45, 'cool_pitch_max':169}}
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
options = {'FPTEMP_11':{'units':'C', 'startlim':-119, 'stoplim':-114, 'limstep':5, 'nccd1':[4,5,6],
                        'dh':['ON', 'OFF'], 'model':'acisfp', 'sameccd':True}}
newcases.extend(assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates))
casechunks = list(chunks(newcases, 14))
for n, cases in enumerate(casechunks):
    write_cases(cases, 'cases_acis_2017-{}.csv'.format(n))
    
dates = [year + ':' + day for day in ['001', '090', '182', '274'] for year in ['2018', ]]
options = {'FPTEMP_11':{'units':'C', 'startlim':-119, 'stoplim':-114, 'limstep':5, 'nccd1':[1,4,5,6],
                        'nccd2':[1,], 'dh':['ON', 'OFF'], 'model':'acisfp', 'sameccd':False,
                        'cool_pitch_min':45, 'cool_pitch_max':169}}
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
options = {'FPTEMP_11':{'units':'C', 'startlim':-119, 'stoplim':-114, 'limstep':5, 'nccd1':[4,5,6],
                        'dh':['ON', 'OFF'], 'model':'acisfp', 'sameccd':True}}
newcases.extend(assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates))
casechunks = list(chunks(newcases, 14))
for n, cases in enumerate(casechunks):
    write_cases(cases, 'cases_acis_2018-{}.csv'.format(n))

In [ ]:
len(newcases)/14.

In [ ]:
cases[1]

-----------

# ACA Model

In [ ]:
n_sim = 2000
max_dwell_ksec = 700
dwell_2_pitch_num = 200
dates = [year + ':' + day for day in ['001', '090', '182', '274'] for year in ['2016', '2017', '2018']]


options = {'AACCCDPT':{'units':'C', 'startlim':-15, 'stoplim':-9, 'limstep':1, 
                       'nccd1':[4,], 'dh':['ON'], 'model':'aca', 'sameccd': True}}

newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
len(newcases)

In [ ]:
casechunks = list(chunks(newcases, 7))
for n, cases in enumerate(casechunks):
    write_cases(cases, 'cases_aca_{}.csv'.format(n))

# DEA Model

In [ ]:
n_sim = 2000
max_dwell_ksec = 700
dwell_2_pitch_num = 200
dates = [year + ':' + day for day in ['001', '090', '182', '274'] for year in ['2016', '2017', '2018']]

options = {'1DEAMZT':{'units':'C', 'startlim':35, 'stoplim':38.0, 'limstep':0.5, 'nccd1':[1, 3], 
                      'dh':['ON', 'OFF'], 'model':'dea', 'sameccd': True}}
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
len(newcases)

In [ ]:
factors(len(newcases))

In [ ]:
336/28.

In [ ]:
casechunks = list(chunks(newcases, 28))
for n, cases in enumerate(casechunks):
    write_cases(cases, 'cases_dea_{}.csv'.format(n))

# DPA Model

In [ ]:
n_sim = 2000
max_dwell_ksec = 700
dwell_2_pitch_num = 200
dates = [year + ':' + day for day in ['001', '090', '182', '274'] for year in ['2016', '2017', '2018']]

options = {'1DPAMZT':{'units':'C', 'startlim':35, 'stoplim':38.0, 'limstep':0.5, 'nccd1':[1, 3], 
                      'dh':['ON', 'OFF'], 'model':'dpa', 'sameccd': True}}
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
len(newcases)

In [ ]:
casechunks = list(chunks(newcases, 28))
for n, cases in enumerate(casechunks):
    write_cases(cases, 'cases_dpa_{}.csv'.format(n))

# Central Cylinder Model

In [13]:
n_sim = 2000
max_dwell_ksec = 700
dwell_2_pitch_num = 200
dates = [year + ':' + day for day in ['001', '090', '182', '274'] for year in ['2017', '2018', '2019']]

options = {'TCYLAFT6':{'units':'F', 'startlim':108, 'stoplim':120, 'limstep':1, 'nccd1':[4,], 'dh':['OFF'],
                       'model':'tcylaft6', 'sameccd': True, 'roll':0.0}}
newcases = assemble_cases(options, n_sim, max_dwell_ksec, dwell_2_pitch_num, dates)
len(newcases)

156

In [14]:
156/12.

13.0

In [15]:
casechunks = list(chunks(newcases, 12))
for n, cases in enumerate(casechunks):
    write_cases(cases, 'cases_cc_{}.csv'.format(n))